In [1]:
import numpy as np
import scipy
import seaborn as sns
import datasets
from datasets import Dataset
import evaluate
import torch
from tqdm.notebook import tqdm
import pickle
import os
os.environ['WANDB_DISABLED'] = 'true'

import utils

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

2023-01-17 02:25:25.234899: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2023-01-17 02:25:25.234943: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [3]:
data = pickle.load(open('training_data.pkl', 'rb'))
data = {'text': [i[0] for i in data], 'label' : [i[1] for i in data]}
dataset = Dataset.from_dict(data)

In [4]:
# train_ds, valid_ds, test_ds = utils.load_dataset()
dataset = dataset.shuffle(seed=0)

In [5]:
'''
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='do_not_pad')

tokenized_ds = ds.map(tokenize_function, batched=True)
lengths = np.array([len(i['input_ids']) for i in tokenized_ds['validation']])
np.quantile(lengths, [0.90]) = 168
'''

'\ntokenizer = AutoTokenizer.from_pretrained("roberta-base")\ndef tokenize_function(examples):\n    return tokenizer(examples["text"], padding=\'do_not_pad\')\n\ntokenized_ds = ds.map(tokenize_function, batched=True)\nlengths = np.array([len(i[\'input_ids\']) for i in tokenized_ds[\'validation\']])\nnp.quantile(lengths, [0.90]) = 168\n'

In [6]:
labels = np.array(dataset['label'])
num_minority = np.sum(labels == 1)

np.random.seed(0)

indexes = []
for label in [0, 1]:
    idx = np.arange(len(labels))
    idx = idx[labels == label]
    
    print(len(idx))
    sample = np.random.choice(idx, size=num_minority, replace=False)
    indexes.append(sample)

100000
4364


In [7]:
indexes = np.concatenate(indexes)
balanced_dataset = dataset.select(indexes)
balanced_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 8728
})

In [8]:
splits = balanced_dataset.train_test_split(test_size=0.1)
train_ds = splits['train']
valid_ds = splits['test']

In [9]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=168, truncation=True)

tokenized_train = train_ds.map(tokenize_function, batched=True)
tokenized_valid = valid_ds.map(tokenize_function, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
np.mean(tokenized_train['label'].numpy())

0.498026734563972

In [11]:
tokenized_valid.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
np.mean(tokenized_valid['label'].numpy())

0.5177548682703322

In [12]:
model = AutoModelForSequenceClassification.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [13]:
training_args = TrainingArguments(output_dir="trainer",
                                  max_steps=1000,
                                  per_device_train_batch_size=32,
                                  evaluation_strategy='steps',
                                  eval_steps=100,
                                  save_strategy='no',
                                  num_train_epochs=1,
                                 )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [14]:
metric = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    batch_scores = scipy.special.softmax(logits, axis=-1)
    return metric.compute(prediction_scores=batch_scores[:,1], references=labels)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [16]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/home/johnny/.conda/envs/torch/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7855
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1000


Step,Training Loss,Validation Loss,Roc Auc
100,No log,0.277721,0.958453
200,No log,0.262655,0.964113
300,No log,0.246393,0.965261
400,No log,0.354772,0.965916
500,0.281000,0.236016,0.969825
600,0.281000,0.300783,0.969347
700,0.281000,0.333815,0.969594
800,0.281000,0.395433,0.960469
900,0.281000,0.430769,0.963829
1000,0.088300,0.417295,0.966000


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 873
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 873
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 873
  Batch size = 8
T

TrainOutput(global_step=1000, training_loss=0.1846529006958008, metrics={'train_runtime': 520.4482, 'train_samples_per_second': 61.485, 'train_steps_per_second': 1.921, 'total_flos': 2756795415857280.0, 'train_loss': 0.1846529006958008, 'epoch': 4.07})

In [17]:
def _evaluate(model, val_dataloader):
    model.eval()
    val_pbar = tqdm(total=len(val_dataloader))

    scores = []
    for batch in val_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch['label']
        del(batch['label'])
        with torch.no_grad():
            outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).cpu().tolist()

        batch_scores = torch.nn.Softmax()(logits)[:,-1]
        scores.append(batch_scores)

        metric.add_batch(prediction_scores=batch_scores, references=labels)
        val_pbar.update(1)
    eval_dict = metric.compute()
    val_pbar.set_description('Roc_auc: %.2f' % eval_dict['roc_auc'])

    eval_dict['predictions'] = torch.cat(scores)

    return eval_dict

In [18]:
#tokenized_test = test_ds.map(tokenize_function, batched=True)
#tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

#val_dataloader = torch.utils.data.DataLoader(tokenized_test, shuffle=False, batch_size=16,# pin_memory=True)
#eval_dict = _evaluate(model, val_dataloader)

In [19]:
#predictions = eval_dict['predictions'].cpu().numpy()
#np.save('scores/roberta_balanced', predictions)

In [20]:
trainer.save_model('./roberta_balanced')

Saving model checkpoint to ./roberta_balanced
Configuration saved in ./roberta_balanced/config.json
Model weights saved in ./roberta_balanced/pytorch_model.bin
